In [244]:
import pandas as pd
import numpy as np
from functools import reduce

In [260]:
# récupère l'input sous forme de DataFrame
with open("input.txt") as file:
    input = file.read().strip().split('\n')
input = [list(line) for line in input]
input = pd.DataFrame(input)
input

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,X,X,X,X,X,U,U,U,N,N,...,K,K,K,K,K,K,K,K,K,K
1,X,X,X,U,X,X,U,U,U,U,...,K,K,K,K,K,K,K,K,K,K
2,X,U,U,U,U,U,U,U,U,U,...,K,K,K,K,K,K,K,K,K,K
3,U,U,U,U,U,U,U,U,U,U,...,K,K,K,K,K,K,K,K,K,G
4,U,U,U,U,U,U,U,U,U,K,...,K,K,K,K,K,K,K,K,K,G
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,T,X,X,X,X,X,X,Y,Y,J,...,T,T,T,T,T,R,R,R,R,R
136,T,X,X,X,X,X,X,X,Y,J,...,T,T,T,T,T,R,R,R,R,R
137,T,X,T,X,X,X,X,X,Y,Y,...,T,T,T,T,R,R,R,R,R,R
138,T,T,T,X,X,X,X,X,X,X,...,T,T,T,R,R,R,R,R,R,R


In [261]:
# réalise une marge tout autour de l'input
input[140] = '#'
input[141] = '#'
input = input[[140] + list(range(140))[0:140] + [141]]
old_col = [140] + list(range(140)) + [141]
new_col = list(range(142))
input.rename(columns = dict(zip(old_col, new_col)), inplace = True)
input = pd.concat([input, pd.DataFrame([142*["#"]])])
input = input.iloc[::-1,:]
input = pd.concat([input, pd.DataFrame([142*["#"]])])
input = input.reset_index(drop=True)
input

,0,1,2,3,4,5,6,7,8,9,...,132,133,134,135,136,137,138,139,140,141
0,#,#,#,#,#,#,#,#,#,#,...,#,#,#,#,#,#,#,#,#,#
1,#,T,T,T,X,X,X,X,X,X,...,R,R,R,R,R,R,R,R,R,#
2,#,T,T,T,X,X,X,X,X,X,...,T,T,R,R,R,R,R,R,R,#
3,#,T,X,T,X,X,X,X,X,Y,...,T,T,T,R,R,R,R,R,R,#
4,#,T,X,X,X,X,X,X,X,Y,...,T,T,T,T,R,R,R,R,R,#
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137,#,U,U,U,U,U,U,U,U,U,...,K,K,K,K,K,K,K,K,G,#
138,#,X,U,U,U,U,U,U,U,U,...,K,K,K,K,K,K,K,K,K,#
139,#,X,X,X,U,X,X,U,U,U,...,K,K,K,K,K,K,K,K,K,#
140,#,X,X,X,X,X,U,U,U,N,...,K,K,K,K,K,K,K,K,K,#


In [262]:
# liste les lettres uniques de l'input
letter_sets = [x for x in[set(input[col].unique()) for col in input]]
global_letter_set = set()
for letter_set in letter_sets:
    global_letter_set = global_letter_set|letter_set
global_letter_list = [x for x in list(global_letter_set) if x is not '#']
global_letter_list

<>:6: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:6: SyntaxWarning: "is not" with a literal. Did you mean "!="?
C:\Users\John\AppData\Local\Temp\ipykernel_5980\3091853525.py:6: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  global_letter_list = [x for x in list(global_letter_set) if x is not '#']


['N',
 'D',
 'Y',
 'G',
 'H',
 'O',
 'I',
 'K',
 'X',
 'C',
 'T',
 'L',
 'W',
 'B',
 'R',
 'S',
 'Z',
 'M',
 'F',
 'Q',
 'P',
 'J',
 'A',
 'V',
 'U',
 'E']

In [263]:
#fonction de regroupement pour une ligne et une lettre donnée 
def idx_to_tuple(idx, liste):
    tuple_list =[]
    for group in liste:
        tuple_group = {(idx,x) for x in group}
        tuple_list.append(tuple_group)
    return tuple_list

def groupage(letter, row):
#localiser les lettres dans une liste par leur indice dans la ligne-liste
    liste = list(np.where(np.array(row)==letter)[0])
    try:
        group_list = [{liste[0]}]
    except IndexError:
        return []
#grouper en sets dans une liste
    for i in range(0,len(liste)-1):
        if liste[i+1] - liste[i]> 1:
            group_list.append({liste[i+1]})
        else:
            group_list[-1] = group_list[-1]|{liste[i+1]}                       
#transformer les éléments en tuple
    return group_list
    
# exemple
idx = 3
row = ['V', 'V', 'I', 'V', 'C', 'V', 'V', 'J', 'E', 'V']
idx_to_tuple(idx, groupage('V', row),)

[{(3, 0), (3, 1)}, {(3, 3)}, {(3, 5), (3, 6)}, {(3, 9)}]

In [264]:
# fonction de rassemblement des groupe-lignes dans un dictionnaire
def group_dict(letter, input):
    #initiate an empty dictionary
    group_dict = {}
    #for row in input
    for idx in range(len(input)):
    #    divide the row into groups of index
        group = groupage(letter,input.iloc[idx,:])
    #    transform each elt into its index tuple
        group = idx_to_tuple(idx, group)
    #    dictionary[row] = group
        group_dict[idx] = group
    #display dictionary
    return group_dict

# exemple
letter = 'R'
input = input
group_dict(letter, input)

{0: [],
 1: [{(1, 68), (1, 69), (1, 70), (1, 71), (1, 72), (1, 73), (1, 74)},
  {(1, 132),
   (1, 133),
   (1, 134),
   (1, 135),
   (1, 136),
   (1, 137),
   (1, 138),
   (1, 139),
   (1, 140)}],
 2: [{(2, 68), (2, 69), (2, 70), (2, 71), (2, 72), (2, 73), (2, 74)},
  {(2, 134), (2, 135), (2, 136), (2, 137), (2, 138), (2, 139), (2, 140)}],
 3: [{(3, 68), (3, 69), (3, 70), (3, 71), (3, 72), (3, 73), (3, 74)},
  {(3, 135), (3, 136), (3, 137), (3, 138), (3, 139), (3, 140)}],
 4: [{(4, 68), (4, 69), (4, 70), (4, 71), (4, 72)},
  {(4, 74)},
  {(4, 136), (4, 137), (4, 138), (4, 139), (4, 140)}],
 5: [{(5, 68), (5, 69), (5, 70), (5, 71), (5, 72)},
  {(5, 136), (5, 137), (5, 138), (5, 139), (5, 140)}],
 6: [{(6, 68), (6, 69), (6, 70), (6, 71), (6, 72), (6, 73), (6, 74)},
  {(6, 139), (6, 140)}],
 7: [{(7, 68),
   (7, 69),
   (7, 70),
   (7, 71),
   (7, 72),
   (7, 73),
   (7, 74),
   (7, 75),
   (7, 76)},
  {(7, 139), (7, 140)}],
 8: [{(8, 67), (8, 68), (8, 69), (8, 70), (8, 71), (8, 72), (8, 

In [265]:
# fonction pour déballer le dictionnaire
def debal_dic(group_dic):
    def add_list (a,b):
        return a+b
    # déballe les groupes du dictionnaire dans une liste
    list_of_lists = [x for x in group_dic.values()]
    unraveled_list = reduce(add_list, list_of_lists)
    return unraveled_list

#exemple
group_dic = {0: [{(0, 4), (0, 5)}],
 1: [{(1, 4), (1, 5)}],
 2: [],
 3: [],
 4: [],
 5: [{(5, 2)}],
 6: [{(6, 2), (6, 3), (6, 4)}],
 7: [{(7, 1), (7, 2), (7, 3), (7, 4), (7, 5)}],
 8: [{(8, 1), (8, 2), (8, 3)}, {(8, 5)}],
 9: [{(9, 3)}]}

debal_dic(group_dic)

[{(0, 4), (0, 5)},
 {(1, 4), (1, 5)},
 {(5, 2)},
 {(6, 2), (6, 3), (6, 4)},
 {(7, 1), (7, 2), (7, 3), (7, 4), (7, 5)},
 {(8, 1), (8, 2), (8, 3)},
 {(8, 5)},
 {(9, 3)}]

In [266]:
# fonction pour joindre deux groupes sur deux lignes distinctes
def group_sets(set_a, set_b):
#si dans a il y a un elt avec un point de différence avec un élément de b
    for tup_a in list(set_a):
        for tup_b in list(set_b):
            if abs(tup_b[0] - tup_a[0]) > 1:
                pass
            elif tup_b[1] - tup_a[1] == 0:
#    retourne l'union de a et b
                return set_a|set_b

#exemple :
set_a = {(8, 6), (8, 2), (8, 3)}
set_b = {(7, 1), (7, 2), (7, 3), (7, 4), (7, 5)}

group_sets(set_a, set_b)

{(7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (8, 2), (8, 3), (8, 6)}

In [267]:
# fonction pour finaliser la jointure des groupes de la liste 
def super_group(unraveled_list):
#    unraveled_list_ = unraveled_lis.copy()
    unraveled_list_ = unraveled_list.copy()
#    for group_a in unraveled_list_:
    for group_a in unraveled_list_:
#        for group_b in unraveled_list_ without group_a :       
        for group_b in [x for x in unraveled_list_ if x != group_a]:
#            if group_a intersects group_b :
            if group_sets(group_a, group_b) is not None: #group_a & group_b_up != set():      
#                unraveled_list_.remove(group_a and group_b)
                try:
                    unraveled_list_.remove(group_a)
                except ValueError:
                    pass
                unraveled_list_.remove(group_b)
#                unraveled_list_.append(union of group_a and group_b)
                unraveled_list_.append(group_a | group_b)
#                super_group(unraveled_list_)
                return super_group(unraveled_list_)

    return unraveled_list_

#exemple :
unraveled_list = [{(0, 4), (0, 5)},
 {(1, 4), (1, 5)},
 {(5, 2)},
 {(6, 2), (6, 3), (6, 4)},
 {(7, 1), (7, 2), (7, 3), (7, 4), (7, 5)},
 {(8, 1), (8, 2), (8, 3)},
 {(8, 5)},
 {(9, 3)}]

ed_list = [{(7, 3)},
 {(8, 3), (8, 4), (8, 5)},
 {(9, 1), (9, 2), (9, 3), (9, 4)},
 {(10, 1), (10, 2), (10, 3), (10, 4)}]

final_list = super_group(ed_list)
final_list

[{(7, 3),
  (8, 3),
  (8, 4),
  (8, 5),
  (9, 1),
  (9, 2),
  (9, 3),
  (9, 4),
  (10, 1),
  (10, 2),
  (10, 3),
  (10, 4)}]

In [268]:
# fonction pour calculer le périmètre d'un groupe
def perimeter(group):
    cadre = {}   
    for i in [-1, 0, 1]:
        for j in [-1, 0, 1]:
            cadre[f'{i}{j}']={(x[0]+i, x[1]+j) for x in list(group)}    
    
    return len(cadre['-10']|group) + len(cadre['10']|group) + len(cadre['0-1']|group) +len(cadre['01']|group) - 4 * len(group)

#exemple
group = {(5,2), (6,2), (6,3),(6,4),(5,4)}
perimeter(group)

12

In [270]:
# init un score
score = 0
# pour chaque lettre
for letter in global_letter_list:
# réalise le dictionnaire des lignes de groupe
    group_dict_ = group_dict(letter, input)
# déballe le dictionnaire
    debal_list_ = debal_dic(group_dict_)
# rassemble les groupes adjacents
    final_list = super_group(debal_list_)
# pour chaque groupe 
    for group in final_list:
# calcule la surface et le périmètre
        surface = len(group)
        périmètre = perimeter(group)
        #print("surface : ", surface,"\npérimètre : ",périmètre)
# ajoute le produit au score
        score += surface * périmètre
# GOOD SCORE : 1465968
score

1465968